# Weakly Compressible Stokes implementation
Riccardo Rossi
UPC BarcelonaTech, CIMNE


the Stokes equations are

$\frac{\partial\rho u}{\partial t}-\nabla\cdot\mathbf{C}\nabla^{s}\mathbf{u}+\nabla p=f$


$\frac{\partial\rho}{\partial t}+k\nabla\cdot\left(\rho\mathbf{\mathbf{u}}\right)=0
\longrightarrow 
\frac{\partial\rho}{\partial p}\frac{\partial p}{\partial t}+\nabla\cdot\left(\rho\mathbf{\mathbf{u}}\right)=0
\longrightarrow 
\frac{1}{k}\frac{\partial p}{\partial t}+\nabla\cdot\left(\rho\mathbf{\mathbf{u}}\right)=0
$

where $\mathbf{u}$ is the velocity, $p$ is the pressure, $\rho$ the density, $\mathbf{C}$ is an appropriate constitutive tensor and $k:=\frac{\partial p}{\partial \rho}$ is the bulk modulus (which may have a non linear dependency on the other parameters)


## Discretization
we consider the following (multiscale) decomposition

$\mathbf{u}=\mathbf{u_{h}}+\mathbf{u}_{s}$

$p=p_{h}+p_{s}$

where $\mathbf{u_{\mathbf{h}}}$ and $p_h$ belong to the finite element space.

If we now apply the Galerkin method and test the initial equations
by $w$ and $q$, we obtain

$\left(\mathbf{w},f-\frac{\partial\rho(\mathbf{u})}{\partial t}+\nabla\cdot\mathbf{C}\nabla^{s}\mathbf{u}-\nabla p\right)+\left(q,-\frac{1}{k}\frac{\partial p}{\partial t}-\nabla\cdot\left(\rho\mathbf{\mathbf{u}}\right)\right)=0$


substituting the proposed decomposition

$\left(\mathbf{w},f-\frac{\partial\rho(\mathbf{u_{h}+\mathbf{u}_{s}})}{\partial t}+\nabla\cdot\mathbf{C}\nabla^{s}\left(\mathbf{u}_{h}+\mathbf{u_{s}}\right)-\nabla(p_{h}+p_{s})\right) 
+\left(q,-\frac{1}{k}\frac{\partial p_h + p_s}{\partial t}-\nabla\cdot\left(\mathbf{\rho u}_{h}\mathbf{+\rho u}_{s}\right)\right) =0
$


we can now focus on the different terms, and integrate by parts as needed to get

$
\left(\mathbf{w},\nabla\cdot C\nabla^{s}\left(\mathbf{u}_{h}+\mathbf{u_{s}}\right)\right)=-\left(\mathbf{\nabla w},C\nabla^{s}\mathbf{u}_{h}\right)+\int_{\Gamma}\mathbf{w}\cdot C\nabla^{s}\mathbf{u}_{h}\mathbf{\cdot n}+  \left(\mathbf{w},2\nabla\cdot\mu\nabla^{s}\mathbf{u_{s}}\right)
$


and

$
\left(\mathbf{w},-\nabla(p_{h}+p_{s})\right) =\left(\mathbf{\nabla\cdot w},p_{h}+p_{s}\right)-\int_{\Gamma}p_{h}\mathbf{w}\mathbf{\cdot n}
$

note that the boundary terms on $\mathbf{u}_s$ $p_{s}$ are dropped after integration by parts since it is __assumed__ that they vanish on the interelement boundaries


similarly, working on the terms corresponding to the mass conservation equation we get

$
\left(q,-\frac{1}{k}\frac{\partial p}{\partial t}
-\nabla\cdot\left(\mathbf{\rho u}_{h}\mathbf{+\rho u}_{s}\right)\right) 
=\left(q,-\frac{1}{k}\frac{\partial p_h}{\partial t}-\nabla\cdot\left(\mathbf{\rho u}_{h}\right)\right)
+\left(\nabla q,\mathbf{\rho u}_{s}\right)
-\left(q,\frac{1}{k}\frac{\partial p_s}{\partial t}\right)
$

adding everything together, the Galerkin discretization becomes 

$
\left(\mathbf{w},f\right)
-\left(\mathbf{w},\frac{\partial\rho\mathbf{u_{h}}}{\partial t}\right)
-\left(\mathbf{\nabla w},C\nabla^{s}\mathbf{u}_{h}\right)
+\int_{\Gamma}\mathbf{w}\cdot2\mu\nabla^{s}\mathbf{u}_{h}\mathbf{\cdot n}
+\left(\mathbf{\nabla\cdot w},p_{h}\right)
+\left(q,-\frac{1}{k}\frac{\partial p_h}{\partial t}-\nabla\cdot\left(\mathbf{\rho u}_{h}\right)\right)
-\int_{\Gamma}\mathbf{w}\mathbf{p_h\cdot n}
$

$+$

$
\left(\mathbf{\nabla\cdot w},p_{s}\right) 
+\left(\nabla q,\mathbf{\rho u}_{s}\right)
-\left(q,-\frac{1}{k}\frac{\partial p_s}{\partial t}\right)
-\left(\mathbf{w},\frac{\partial\rho\mathbf{u_{s}}}{\partial t}\right)
$

### Introducing ASGS multiscale **quasi-static** model

We now need to introduce a model for the subscales (**ASGS**)

 $\rho \mathbf{u}_{s}=\tau_{1}\left(\mathbf{f}-\frac{\partial\rho\mathbf{u}_{\mathbf{h}}}{\partial t}-\nabla p_{h}\right)$ **NOTE:** we are modelling $\rho \mathbf{u}_s$ not just $\mathbf{u}_s$

$p_{s}=\tau_{2}\left(-\frac{\partial p_h}{\partial t}-k\nabla\cdot\left(\mathbf{\rho u}_{h}\right)\right)$

It is customary to assume that the time step chosen in the simulation is sufficiently large to consider
as quasi-static the response of the subgrid-scale terms. For this reason, 
we will make the approximation $\frac{\partial\rho \mathbf{u}_{s}}{\partial t}\approx\mathbf{0}$ and $\frac{\partial p_s}{\partial t}\approx\mathbf{0}$ 
(quasi-static subscales) 

We remark here that if we use linear elements, all higher-than-one order derivatives of finite element functions within each element can be discarded, which also justifies why the viscous term was neglected in the model of the subscale velocity


$\mathbf{\left(\mathbf{w},\nabla\cdot\mathbf{C}\nabla^{s}\mathbf{u_{s}}\right)}=-\mathbf{\left(\mathbf{\nabla w},\mathbf{C}\nabla^{s}\mathbf{u_{s}}\right)+\int_{\Gamma}...}\approx0$


The final, stabilized, form is obtained by replacing the subscale model into the discrete galerkin equation

$\left(\mathbf{\nabla\cdot w},p_{s}\right)\rightarrow \left(\nabla\cdot\mathbf{w},\tau_{2}\left(-\frac{1}{k}\frac{\partial p_h}{\partial t}-\nabla\cdot\left(\mathbf{\rho u}_{h}\right)\right)\right)$

Pressure Equation Terms

$\left(\nabla q,\rho\mathbf{u_{s}}\right)\rightarrow\left(\nabla q,\tau_{1}\left(\mathbf{f}-\frac{\partial\rho\mathbf{u}_{h}}{\partial t}-\nabla p_{h}\right)\right)$


## SYMBOLS TO BE EMPLOYED
Shape functions $N_{I}$ and derivatives $\nabla N_{I}$stored respectively
in a vector $\mathbf{N}$and a matrix $\mathbf{DN}$


we also introduce the constitutive matrix $\mathbf{C}$(symmetric
6x6 in 3D) and the matrix $\mathbf{B}$ (6x12 for a tetra) such
that $\sigma=\mathbf{B}\epsilon$

In [68]:
from sympy import *

import sys
sys.path.append("../../../../kratos/python_scripts/")

from KratosMultiphysics import *
from sympy_fe_utilities import *

do_simplifications = False
dim = 3 #spatial dimensions
mode = "c" #to output to a c++ file

if(dim == 2):
    nnodes = 3
    strain_size = 3
else:
    nnodes = 4
    strain_size = 6   


impose_partion_of_unity = False
N,DN = DefineShapeFunctions(nnodes, dim, impose_partion_of_unity)

#defining the unknowns
v = DefineMatrix('v',nnodes,dim)     #v(i,j) is velocity of node i component j
vn = DefineMatrix('vn',nnodes,dim)   #velocity one step back
vnn = DefineMatrix('vnn',nnodes,dim) #velocity two step back
p = DefineVector('p',nnodes)         #p[i] is the pressure of node i at the current step
pn = DefineVector('pn',nnodes)       #old step pressures
pnn = DefineVector('pnn',nnodes)     #pressures two step back
r = DefineVector('r',nnodes)         #r[i] = density of node i on the current step
rn = DefineVector('rn',nnodes)       #rn[i] = density of node i one step back
rnn = DefineVector('rnn',nnodes)     #rn[i] = density of node i one step back
k = Symbol("k")                      #bulk modulus on gauss

#define test functions
w = DefineMatrix('w',nnodes,dim)     #w[i,j] = velocity test function, node i, component j
q = DefineVector('q',nnodes)         #p[i] = pressure test function, node i

#define other nodally varying data
f = DefineMatrix('f',nnodes,dim)     #f[i,j] = force, node i, component j

#constitutive matrix
C = DefineSymmetricMatrix('C',strain_size,strain_size) #constitutive tensor in Voigt Format
    
#define other symbols
tau1 = Symbol('tau1', positive=True) 
tau2 = 0.0 #Symbol('tau2')

bdf0 = Symbol('bdf0') #coefficients used by the time scheme
bdf1 = Symbol('bdf1')
bdf2 = Symbol('bdf2')

#computing gradients
grad_v = DN.transpose()*v #gradient of velocity
grad_q = DN.transpose()*q #gradient of the pressure test function
grad_p = DN.transpose()*p  #gradient of pressure

#here we define a variable "stress" which will be used as such in the calculation of the RHS
#when calculating the LHS, we will assume that stress = C*strain
if(dim == 2):
    stress = DefineVector('stress',3) 

elif(dim == 3):
    stress = DefineVector('stress',6)

B = MatrixB(DN)
grad_sym_f = grad_sym_voigtform(DN,f)

rho_v = DefineMatrix('rho_v',nnodes,dim)     #auxiliary value containing rho*v
DrhovDt = DefineMatrix('DrhovDt',nnodes,dim) #contains d(rho*v)/dt
DpDt = DefineVector('DpDt',nnodes)           #contains dp/dt
for i in range(nnodes):
    DpDt[i] = bdf0*p[i]+bdf1*pn[i]+bdf2*pnn[i]
    for j in range(dim):
        rho_v[i,j] = r[i]*v[i,j]
        DrhovDt[i,j] = bdf0*r[i]*v[i,j]+bdf1*rn[i]*vn[i,j]+bdf2*rnn[i]*vnn[i,j]

div_rhov = div(DN,rho_v)               #divergence of rho*v
div_w = div(DN,w)                      #divergence of the test function w
grad_sym_w = grad_sym_voigtform(DN,w)  #symmetric gradient of w (in voigt form)

#values interpolated on the gauss point
DrhovDt_gauss = DrhovDt.transpose()*N  
DpDt_gauss = DpDt.transpose()*N 
wgauss = w.transpose()*N 
qgauss = q.transpose()*N
pgauss = p.transpose()*N
fgauss = f.transpose()*N

vector_of_ones = DefineVector('vector_of_ones',dim)
for i in range(dim):
    vector_of_ones[i] = 1.0
    


## IMPLEMENTATION - GALERKIN PART
The term

$\left(\mathbf{w},f-\frac{\partial\rho u_{h}}{\partial t}\right)
-\left(\nabla^s\mathbf{w},\mathbf{\sigma}\right)
+\left(\nabla\cdot \mathbf{w},p_{h}\right)
+
\left(q,-\frac{1}{k}\frac{\partial p_h}{\partial t}-\nabla\cdot\left(\mathbf{\rho u}_{h}\right)\right)
$

Is implemented on each gauss point as

In [69]:
rv_galerkin =  wgauss.transpose()*(fgauss) \
    - wgauss.transpose()*DrhovDt_gauss  \
    - grad_sym_w.transpose()*stress +  div_w*pgauss        \
    - qgauss*(div_rhov) \
    - 1/k*qgauss.transpose()*DpDt_gauss
    



## IMPLEMENTATION-STABILIZATION PART

The stabilization terms to be implemented are:

$
\left(\nabla\cdot\mathbf{w},\tau_{2}\left(-\frac{1}{k}\frac{\partial p_h}{\partial t}-\nabla\cdot\left(\mathbf{\rho u}_{h}\right)\right)\right) 
+
\left(\nabla q,\tau_{1}\left(\mathbf{f}-\frac{\partial\rho\mathbf{u}_{h}}{\partial t}-\nabla p_{h}\right)\right)$


which is implemented as

In [70]:
rv_stab = div_w*tau2*(-1/k*DpDt_gauss-div_rhov) \
        +  grad_q.transpose()*(tau1*(fgauss - DrhovDt_gauss - grad_p) ) 
    
##TOTAL RESIDUAL
rv = rv_galerkin + rv_stab


## DEFINE TEST FUNCTIONS AND DOFS AND COMPTUE THE FEM RHS

In [71]:

#define dofs & test function vector
dofs = Matrix( zeros(nnodes*(dim+1), 1) )
testfunc = Matrix( zeros(nnodes*(dim+1), 1) )
for i in range(0,nnodes):
    for k in range(0,dim):
        dofs[i*(dim+1)+k] = v[i,k]
        testfunc[i*(dim+1)+k] = w[i,k]
    dofs[i*(dim+1)+dim] = p[i,0]
    testfunc[i*(dim+1)+dim] = q[i,0]
print("dofs = ",dofs)

#here we derive the functional by the test functions, thus computing the FEM RHS
rhs = Compute_RHS(rv, testfunc, do_simplifications)


dofs =  Matrix([[v_0_0], [v_0_1], [v_0_2], [p_0], [v_1_0], [v_1_1], [v_1_2], [p_1], [v_2_0], [v_2_1], [v_2_2], [p_2], [v_3_0], [v_3_1], [v_3_2], [p_3]])


## COMPUTE LHS

In order to compute the LHS by differentiation, we explicitly introduce the dependency 
of the stress on the strain (symmetric gradient of the velocity). This way of proceeding is
needed in order to take into account nonlinear material behaviours.

In [72]:
strain = grad_sym_voigtform(DN,v)

#here we substitute "stress" by "C*strain"
rhs_to_derive = SubstituteMatrixValue( rhs.copy(), stress, C*strain )

##obtain LHS by symbolic derivation
lhs = Compute_LHS(rhs_to_derive, testfunc, dofs, do_simplifications)


# Expression employed for the stabilization coefficients

we employ the shape function gradients in the estimation of the inverse of h. The idea here is that the norm of the gradient of a shape functions is (for a simplex element) the inverse of the height associated to the node of interest

In [73]:
#simplified way of computing tau
if(dim == 2):
    mu_eq = C[2,2]
elif(dim == 3):
    mu_eq = (C[3,3]+C[4,4]+C[5,5])/3
    
inv_h2 = 0
for i in range(nnodes):
    for j in range(dim):
        inv_h2 += DN[i,j]**2
    
tau_denom = 3*mu_eq*inv_h2

## CODE GENERATION

here we do the code generation by substituting into the template file

In [74]:
templatefile = open("stokes_weakly_compressible_cpp_template_3D.cpp")
outstring=templatefile.read()

outstring = outstring.replace("//substitute_lhs",   OutputMatrix_CollectingFactors(lhs,"lhs",mode) )
outstring = outstring.replace("//substitute_rhs", OutputVector_CollectingFactors(rhs,"rhs",mode))
outstring = outstring.replace("//replace_tau_denom", OutputSymbolicVariable(tau_denom,mode) )

out = open("stokes_3D_weakly_compressible.cpp",'w')
out.write(outstring)

out.close()